In [8]:
import numpy as np
import pandas as pd

In [9]:

train = pd.read_csv('Desktop/emoji.csv',header=None)
test = pd.read_csv('Desktop/emoji_test.csv',header=None)

In [42]:
train.head()

,0,1,2,3
0,"[never, talk, to, me, again]",3,NaN,NaN
1,"[I, am, proud, of, your, achievements]",2,NaN,NaN
2,"[It, is, the, worst, day, in, my, life]",3,NaN,NaN
3,"[Miss, you, so, much]",0,NaN,[0]
4,"[food, is, life]",4,NaN,NaN


In [15]:
import emoji as emoji

In [16]:

emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [17]:
emoji.emojize(":fork_and_knife:")

'🍴'

In [18]:

for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [22]:
data = train.values
for i in range(10):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [23]:
from keras.utils import to_categorical

In [24]:
XT = train[0]
Xt = test[0]

YT = to_categorical(train[1])
Yt = to_categorical(test[1])


print(XT.shape)
print(Xt.shape)
print(YT.shape)
print(Yt.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [44]:
embeddings = {}
with open('Documents/glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word] = coeffs

In [46]:
embeddings['the'].shape

(50,)

In [26]:
def getOutputEmbeddings(X):
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [27]:

emb_XT = getOutputEmbeddings(XT)
emb_Xt = getOutputEmbeddings(Xt)


c:\users\piyush\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [28]:
print(emb_XT.shape)
print(emb_Xt.shape)

(132, 10, 50)
(56, 10, 50)


In [30]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential

In [31]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 64)            29440     
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
____________________________________________________

In [33]:

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(emb_XT,YT,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

Train on 118 samples, validate on 14 samples
Epoch 1/40
118/118 [==============================] - 6s 49ms/sample - loss: 1.5916 - acc: 0.2203 - val_loss: 1.6188 - val_acc: 0.1429
Epoch 2/40
118/118 [==============================] - 0s 507us/sample - loss: 1.5257 - acc: 0.2966 - val_loss: 1.6334 - val_acc: 0.2143
Epoch 3/40
118/118 [==============================] - 0s 473us/sample - loss: 1.4860 - acc: 0.2797 - val_loss: 1.6531 - val_acc: 0.2857
Epoch 4/40
118/118 [==============================] - 0s 456us/sample - loss: 1.4443 - acc: 0.3475 - val_loss: 1.6013 - val_acc: 0.2857
Epoch 5/40
118/118 [==============================] - 0s 456us/sample - loss: 1.4062 - acc: 0.3475 - val_loss: 1.5183 - val_acc: 0.4286
Epoch 6/40
118/118 [==============================] - 0s 499us/sample - loss: 1.3511 - acc: 0.4237 - val_loss: 1.4185 - val_acc: 0.5000
Epoch 7/40
118/118 [==============================] - 0s 507us/sample - loss: 1.2506 - acc: 0.5593 - val_loss: 1.3187 - val_acc: 0.5000
Epoc

In [35]:

model.evaluate(emb_Xt,Yt)

56/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[1.3987882477896554, 0.5714286]

In [37]:
pred = model.predict_classes(emb_Xt)

In [39]:
pred

array([4, 3, 2, 0, 2, 2, 3, 2, 4, 2, 1, 2, 0, 3, 1, 3, 2, 2, 1, 2, 3, 0,
       4, 0, 3, 3, 2, 0, 1, 2, 0, 1, 0, 2, 0, 1, 2, 3, 4, 2, 1, 0, 0, 1,
       2, 2, 2, 2, 0, 1, 1, 0, 3, 2, 2, 0], dtype=int64)

In [40]:
for i in range(30):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
😁
she got me a present
❤️
❤️
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
😁
I am upset
❤️
😓
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
🍴
Stop making this joke ha ha ha
😁
😁
where is the ball
⚾
⚾
work is hard
😓
😁
This girl is messing with me
😓
❤️
are you serious ha ha
😁
😓
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😁
Congratulation for having a baby
😁
😁
stop messing around
😓
⚾
any suggestions for dinner
🍴
😁
I love taking breaks
❤️
😓
you brighten my day
😁
❤️
I boiled rice
🍴
🍴
she is a bully
😓
❤️
Why are you feeling bad
😓
😓
I am upset
😓
😓
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
⚾
valentine day is near
❤️
😁
